In [2]:
import scanpy as sc 
import anndata as ad
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import os

import datetime


In [4]:
#Define constants here 
#Current date format is YYYYMMDD with automatically updated date
today = datetime.datetime.now().strftime("%Y%m%d")

In [3]:
def read_10x_output(file_path, smp_list, metadata=None, type = 'raw', umi_filter=5):
    import os
    
    #Writing output from separate samples, processed using CellRanger, into a dictionary of Scanpy objects:
    ad = {}

    #Generate AnnData for each sample
    for sample_name in smp_list:
        path = file_path + sample_name
        for i in os.listdir(path):
            if type in i and 'h5' in i:
                file = i
        ad[sample_name] = sc.read_10x_h5(file_path + sample_name +'/'+file)
        ad[sample_name].var.rename(columns = {'gene_ids':'ENSEMBL'}, inplace = True)
        ad[sample_name].var['SYMBOL'] = ad[sample_name].var.index
        ad[sample_name].var.index = ad[sample_name].var['ENSEMBL']
        ad[sample_name].var.drop(columns=['ENSEMBL'], inplace=True)
        #ad[sample_name].var_names_make_unique() 
        
        
        sc.pp.calculate_qc_metrics(ad[sample_name], inplace=True)
        ad[sample_name] = ad[sample_name][ad[sample_name].obs['total_counts'] > umi_filter, :]
        ad[sample_name].var['mt'] = [gene.startswith('mt-') 
                                     for gene in ad[sample_name].var['SYMBOL']]
        ad[sample_name].obs['mt_frac'] = (ad[sample_name][:, 
               ad[sample_name].var['mt'].tolist()].X.sum(1).A.squeeze() 
                                          / ad[sample_name].obs['total_counts'])
        
        ad[sample_name].obs['sample'] = sample_name
        ad[sample_name].obs['barcode'] = ad[sample_name].obs_names
        ad[sample_name].obs_names = ad[sample_name].obs['sample']+"_"+ad[sample_name].obs['barcode']

    #Merge AnnData objects from all the samples together    
    from scipy.sparse import vstack
    stack = vstack([ad[x].X for x in smp_list]) # stack data
    adata = sc.AnnData(stack, var = ad[smp_list[0]].var)
    adata.obs = pd.concat([ad[x].obs for x in smp_list], axis = 0)

    if metadata is not None:
        #Add cleaned metadata to the Anndata.obs table
        obs_merged = pd.merge(left = adata.obs, right = metadata, 
                              how = "left", left_on="sample", right_on="sample")
        obs_merged.index = obs_merged['sample']+"_"+obs_merged['barcode']
        print(obs_merged.index.equals(adata.obs.index))
        adata.obs = obs_merged

    return adata, ad


In [4]:
#Create a sample list in form of a dataframe to be concatenated by the read_10x_output function
sample_list = pd.DataFrame({
    'sample': ['cellranger710_count_05e682d5679826b9b76d6bec731bbe61']
})

In [ ]:
#Save results in the following folder (create if it doesn't exist)

results_folder = '/nfs/team227/kl11/'
if not os.path.exists(results_folder):
    os.makedirs(results_folder)
    
sc_data_folder = '/nfs/team205/vk7/sanger_projects/cell2state_tf_activation/data/pilot_5_prime_dec_2022/'
adata, ad_list = read_10x_output(
    sc_data_folder, 
    smp_list=sample_list['sample'],
    metadata=sample_list, type = 'raw')
# export aggregated and pre-processed data
adata.write(f'{results_folder}{today}_all_cells_with_empty.h5ad')

adata, ad_list = read_10x_output(
    sc_data_folder, 
    smp_list=sample_list['sample'],
    metadata=sample_list, type = 'filtered')
# export aggregated and pre-processed data
adata.write(f'{results_folder}{today}_all_cells.h5ad')